<a href="https://colab.research.google.com/github/jefelder/MLCourse_Project1/blob/FinalProject_ChickenStrike/ChickenStrike_TemplateMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

#############################################################################
############### Function to rotate an image by a given angle ################
#############################################################################
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated, M

def get_rotated_corners(w, h, M):
    corners = np.array([
        [0, 0],
        [w, 0],
        [w, h],
        [0, h]
    ])
    ones = np.ones(shape=(len(corners), 1))
    points_ones = np.hstack([corners, ones])
    rotated_corners = M.dot(points_ones.T).T
    return rotated_corners

def match_template_and_get_corners(image, templates, scales, angles, threshold):

  # Initialize a list to hold the corner coordinates of the matching regions
  all_scores = []
  matching_corners = []

  for template in templates:
    for scale in scales:
      # Resize template
      w, h = template.shape[::-1]
      resized_template = cv2.resize(template, (int(w*scale), int(h*scale)))
      #resized_height, resized_width = resized_template.shape
      for angle in angles:
        # Rotate the template image
        rotated_template, M = rotate_image(resized_template, angle)

        # Perform template matching
        res = cv2.matchTemplate(image, rotated_template, cv2.TM_CCOEFF_NORMED)
        # Find locations where the matching result exceeds the threshold
        loc = np.where(res >= threshold)

        # Get the width and height of the rotated template
        h, w = rotated_template.shape

        # Get the corners of the rotated template
        rotated_corners = get_rotated_corners(w, h, M)

        # Loop through the detected locations
        for pt in zip(*loc[::-1]):
            all_scores.append(res[pt[1], pt[0]])
            # Calculate the coordinates of each corner of the matching region
            adjusted_corners = []
            for corner in rotated_corners:
                adjusted_x = corner[0] + pt[0]
                adjusted_y = corner[1] + pt[1]
                adjusted_corners.append((adjusted_x, adjusted_y))

            matching_corners.append(adjusted_corners)
  # print("max chicken confidence: ", np.max(res))
  return all_scores, matching_corners

# Function to compute IoU for rotated bounding boxes
def rotated_iou(boxA, boxB):
    # Convert box points to a format suitable for cv2.rotatedRectangleIntersection
    # returns smallest rectangle (not necessarily axis-aligned) that encloses points of each box
    # each rect has center, width, height, and rotation angle

    rect1 = cv2.minAreaRect(np.array(boxA, dtype=np.float32))
    rect2 = cv2.minAreaRect(np.array(boxB, dtype=np.float32))
    intersection = cv2.rotatedRectangleIntersection(rect1, rect2)[1]

    if intersection is None:
      return 0.0

    # converts convex hull of intersection points to get closed polygon
    int_pts = cv2.convexHull(intersection, returnPoints=True)
    # calculates area of polygon = area of intersection
    inter_area = cv2.contourArea(int_pts)
    # calcuate area of orignial boxes to use in iou calc
    area_A = cv2.contourArea(np.array(boxA, dtype=np.float32))
    area_B = cv2.contourArea(np.array(boxB, dtype=np.float32))

    iou = inter_area / (area_A + area_B - inter_area)
    return iou

#############################################################################
############### Rotated bb non-max suppresion ################
#############################################################################
def rotated_non_max_suppression(boxes, scores, iou_threshold):
    # indices contains scores in descending order
    indices = np.argsort(scores)[::-1]

    keep = []

    while len(indices) > 0:
        current = indices[0]
        keep.append(current) # keep contains the score
        remove_indices = [0]

        # print("keep ", keep)

        for i in range(1, len(indices)):
            iou = rotated_iou(boxes[current], boxes[indices[i]])
            if iou > iou_threshold:
                remove_indices.append(i)

        # print("removed indicies: ", remove_indices)
        indices = np.delete(indices, remove_indices)
        # print("indicies after: ", indices)

    return keep

############ draw rectangles ######################################
def draw_rotated_rect(img, corners, color=(0, 255, 0)):
    corners = np.int0(corners)
    cv2.drawContours(img, [corners], 0, color, 2)
###################################################################

################# Perform Template Matching on Feeder #######################
# Define function to perform template matching and find the best score
def find_best_match(img_gray, feeder_templates, feeder_scales):
    all_feed_res = []
    all_sizes = []

    # Iterate through each feeder template
    for feeder_template in feeder_templates:
        for scale in feeder_scales:
            # Resize template
            feed_w, feed_h = feeder_template.shape[::-1]
            resized_feed_template = cv2.resize(feeder_template, (int(feed_w*scale), int(feed_h*scale)))
            all_sizes.append(resized_feed_template.shape)

            # Perform template matching
            feed_res = cv2.matchTemplate(img_gray, resized_feed_template, cv2.TM_CCOEFF_NORMED)
            all_feed_res.append(feed_res)

    # Initialize variables to track the best match
    best_max_val = -np.inf
    best_max_loc = None
    best_size = None

    # Loop over all template matching results to find the best one
    for idx, feed_res in enumerate(all_feed_res):
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(feed_res)
        if max_val > best_max_val:
            best_max_val = max_val
            best_max_loc = max_loc
            best_size = all_sizes[idx]
        # print("max_loc: ", max_loc)
        # draw max_loc points
        # cv2.circle(img_rgb, max_loc, radius=5, color=(0,0,255), thickness=-1)
    return best_max_loc, best_size
##############################################################################

# Upload main image files
# image_folder_path = '/content/drive/MyDrive/Object Detection/Chickens/test_images'
# image_filenames = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# # Read file names
# images = [cv2.imread(name) for name in image_filenames]

img_rgb = cv2.imread('/content/drive/MyDrive/Object Detection/Chickens/test_images/chicken_group_4.png')
# resize image to prevent issues caused by screenshot scaling
common_width = 3647
common_height = 1820
#img_rgb = [cv2.resize(cv2.imread(name, 0), (common_width, common_height)) for name in images]
img_rgb = cv2.resize(img_rgb, (common_width, common_height))

# # Upload template images
template_folder_path = '/content/drive/MyDrive/Object Detection/Chickens/template_images'

# List the filenames in the specified directory
template_filenames = [os.path.join(template_folder_path, f) for f in os.listdir(template_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Read file names
templates = [cv2.imread(name, 0) for name in template_filenames]

#angles = [0, 15, 25, 35, 45, 55, 65, 75, 85, 115, 130, 145]
angles = [0, 15, 30]
# scales = [1, 1.25, 1.5, 1.75]
scales = [0.75, 1]

# Perform template matching
# for img_rgb in images:
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
# Perform template matching and get the corner coordinates of the matching regions
all_scores, matching_corners = match_template_and_get_corners(img_gray, templates, scales, angles, threshold=0.67)
# ################### Feeder Components ###################
# Load and resize feeder templates

feeder_template_folder_path = '/content/drive/MyDrive/Object Detection/Chickens/feeder_templates'
feeder_template_filenames = [os.path.join(feeder_template_folder_path, f) for f in os.listdir(feeder_template_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

common_width = 405
common_height = 295
feeder_templates = [cv2.resize(cv2.imread(name, 0), (common_width, common_height)) for name in feeder_template_filenames]

# Define the scales to use
feeder_scales = [0.75, 1, 1.5]

############## Perform NMS #########################
# Convert lists to numpy arrays
matching_corners = np.array(matching_corners)
all_scores = np.array(all_scores)

# Apply non-maximum suppression.
nms_boxes = rotated_non_max_suppression(matching_corners, all_scores, 0.95)

#Draw the remaining bounding boxes on the image
for idx in nms_boxes:
    draw_rotated_rect(img_rgb, matching_corners[idx])

################### Identify Feeder #################################

  # Find the best match for the feeder template
    best_max_loc, best_size = find_best_match(img_gray, feeder_templates, feeder_scales)

    if best_max_loc is not None and best_size is not None:
        # Calculate oval center
        feeder_template_height, feeder_template_width = best_size
        center_x = best_max_loc[0] + feeder_template_width // 2
        center_y = best_max_loc[1] + feeder_template_height // 2

        # Calculate axis lengths assuming perfect ellipse
        axes_lengths = (feeder_template_width // 2, feeder_template_height // 2)

        # Assume the angle of the ellipse in the template image is 0
        feed_angle = 0

        # Draw the detected ellipse on the source image
        cv2.ellipse(img_rgb, (center_x, center_y), axes_lengths, feed_angle, 0, 360, (0, 255, 0), 2)

# Display the result
cv2_imshow(img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Check areas of overlap
def create_mask(shape, contours=None, ellipse_params=None):
    """
    Create a binary mask with drawn contours or ellipse.

    Parameters:
    - shape: tuple, shape of the mask (height, width)
    - contours: list of contours to be drawn on the mask
    - ellipse_params: tuple with parameters for drawing ellipse (center, axes, angle)

    Returns:
    - mask: numpy array with drawn contours or ellipse
    """
    mask = np.zeros(shape, dtype=np.uint8)
    if contours is not None:
        contours = np.array(contours, dtype=np.int32)
        cv2.drawContours(mask, [contours], -1, (255), thickness=cv2.FILLED)
    if ellipse_params is not None:
        center, axes, angle = ellipse_params
        cv2.ellipse(mask, center, axes, angle, 0, 360, (255), thickness=cv2.FILLED)
    return mask

def check_overlap(mask1, mask2):
    """
    Check if there is an overlap between two binary masks.

    Parameters:
    - mask1: numpy array, first binary mask
    - mask2: numpy array, second binary mask

    Returns:
    - overlap: bool, True if there is an overlap, False otherwise
    """
    overlap = np.any(np.bitwise_and(mask1, mask2))
    return overlap

mask_shape = img_gray.shape

feeder_mask = create_mask(mask_shape, ellipse_params=((center_x, center_y), axes_lengths, feed_angle))

chicken_feeding = 0
chicken_not_feeding = 0

for idx in nms_boxes:
  chicken_mask = create_mask(mask_shape, contours=matching_corners[idx])
  if check_overlap(chicken_mask, feeder_mask):
    chicken_feeding += 1
  else:
    chicken_not_feeding += 1

print("Chickens feeding: ", chicken_feeding)
print("Chickens not feeding: ", chicken_not_feeding)